# Topic Modeling (Prepare)

On Monday we talked about summarizing your documents using just token counts. Today, we're going to learn about a much more sophisticated approach - learning 'topics' from documents. Topics are a latent structure. They are not directly observable in the data, but we know they're there by reading them.

> **latent**: existing but not yet developed or manifest; hidden or concealed.

## Use Cases
Primary use case: what the hell are your documents about? Who might want to know that in industry - 
* Identifying common themes in customer reviews
* Discovering the needle in a haystack 
* Monitoring communications (Email - State Department) 

## Learning Objectives
*At the end of the lesson you should be able to:*
* Part 0: Warm-Up
* Part 1: Describe how an LDA Model works
* Part 2: Estimate a LDA Model with Gensim
* Part 3: Interpret LDA results & Select the appropriate number of topics

# Part 0: Warm-Up
How do we do a grid search? 

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [8]:
data = fetch_20newsgroups()

In [9]:
# create X and Y from data
X = data.data
Y = data.target

In [10]:
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,1))
rfc = RandomForestClassifier()

In [6]:
pipe = Pipeline([("vect", tfidf),
                 ("clf", rfc)])

In [7]:
X_dtm = tfidf.fit_transform(X)

### GridSearch on Just Classifier
* Fit the vectorizer and prepare BEFORE it goes into the gridsearch

In [8]:
# create tfidf instant

# transform data

# create parameter dict
parameters = {
    "n_estimators":[10,100],
    "max_depth" : [None, 5, 10]
}
# create model instance 
gs = GridSearchCV(rfc, parameters, cv=3, n_jobs=-1, verbose=1)
# instantiate a grid search object

# optimize model parameters


In [9]:
gs.fit(X_dtm, Y)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [ ]:
# create and tranform a sample text 
my_doc = ["my pet dog isnt real."]
# use the model to classify the sample tet 
my_doc_dtm = tfidf.transform(my_doc)

In [ ]:
gs.predict(my_doc_dtm)

### GridSearch with BOTH the Vectoizer & Classifier

In [ ]:
from sklearn.pipeline import Pipeline

# create tfidf instance
v2 = TfidfVectorizer()
# create model instance 
clf = RandomForestClassifier()
# Create a pipline instance using tfidf and your model 
pipe = Pipeline ([
    ('vect', v2),
    ('clf', clf)
])
# create parameter dict
p2 = {'vect__max_features': (500, 1000),
'clf__n_estimators':(10, 100),
    'clf__max_depth':(15, 20)

}
# instantiate a grid search object
gs = GridSearchCV(pipe, p2, cv=3, n_jobs=-1, verbose=1)
# optimize model parameters


# YOUR CODE HERE
gs.fit(X, Y)

In [ ]:
# pass a raw text sample into the optimized model in order to classify it

In [ ]:
gs.predict(my_doc)

Advantages to using GS with the Pipe:
* Allows us to make predictions on raw text increasing reproducibility. :)
* Allows us to tune the parameters of the vectorizer along side the classifier. :D 

# Part 1: Describe how an LDA Model works

[Your Guide to Latent Dirichlet Allocation](https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d)

[LDA Topic Modeling](https://lettier.com/projects/lda-topic-modeling/)


In [11]:
# Download spacy model
import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [12]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
df = pd.DataFrame({
    'content': data['data'],
    'target': data['target'],
    'target_names': [data['target_names'][i] for i in data['target']]
})
df

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space
...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13,sci.med
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,4,comp.sys.mac.hardware
11311,From: westes@netcom.com (Will Estes)\nSubject:...,3,comp.sys.ibm.pc.hardware
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1,comp.graphics


In [14]:
# can visit (https://regex101.com/) for a regex python editor (has cheat sheets)

# Remove whitespace
df['content'] = df['content'].apply(lambda text: text.strip())
# Remove Emails
df['content'] = df['content'].apply(lambda text: re.sub(r'From: \S+@\S+', "", text))
# Remove new line characters
df['content'] = df['content'].apply(lambda text: re.sub('\n+', "", text))
# Remove non- alphanumeric characters
df['content'] = df['content'].apply(lambda text: re.sub("[^0-9 a-z-A-Z]+", '', text))
# YOUR CODE HERE


In [15]:
df.head()

,content,target,target_names
0,wheres my thingSubject WHAT car is thisNntp-P...,7,rec.autos
1,Guy KuoSubject SI Clock Poll - Final CallSumm...,4,comp.sys.mac.hardware
2,Thomas E WillisSubject PB questionsOrganizati...,4,comp.sys.mac.hardware
3,Joe GreenSubject Re Weitek P9000 Organization...,1,comp.graphics
4,Jonathan McDowellSubject Re Shuttle Launch Qu...,14,sci.space


In [16]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

ValueError: cannot find context for 'fork'

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [76]:
# create our tokens in the form of lemmas 
%time
df['lemmas'] = df['content'].parallel_apply(lambda x: [token.lemma_ for token in nlp(x) if (token.is_stop != True) and (token.is_punct != True)])

Wall time: 0 ns
C:\Users\megal\anaconda3\envs\u4-s1-nlp\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: 'Series' object has no attribute 'parallel_apply'

### The two main inputs to the LDA topic model are the dictionary (id2word) and the corpus.

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(documents=df["lemmas"])
# Term Document Frequency
id2word.doc2bow(doc)
# stores (token id, token count) for each doc in the corpus

# Human readable format of corpus (term-frequency)


# YOUR CODE HERE
raise NotImplementedError()

# Part 2: Estimate a LDA Model with Gensim

 ### Train an LDA model

In [ ]:
### This cell runs the single-processor version of the model (slower)
# %%time
# lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                            id2word=id2word,
#                                            num_topics=20, 
#                                            chunksize=100,
#                                            passes=10,
#                                            per_word_topics=True)
# lda_model.save('lda_model.model')
# # https://radimrehurek.com/gensim/models/ldamodel.html

In [ ]:
### This cell runs the multi-processor version of the model (faster)
%%time
lda_multicore = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                        id2word=id2word,
                                                        num_topics=20, 
                                                        chunksize=100,
                                                        passes=10,
                                                        per_word_topics=True,
                                                        workers=12)


lda_multicore.save('lda_multicore.model')
# https://radimrehurek.com/gensim/models/ldamulticore.html

In [ ]:
from gensim import models
#lda_multicore =  models.LdaModel.load('lda_multicore.model')

### View the topics in LDA model

In [ ]:
# display the print_topics from the model 

# Part 3: Interpret LDA results & Select the appropriate number of topics

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_multicore, corpus, id2word)
vis

### What is topic coherence?
Topic Coherence measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic. These measurements help distinguish between topics that are semantically interpretable topics and topics that are artifacts of statistical inference.
A set of statements or facts is said to be coherent, if they support each other. Thus, a coherent fact set can be interpreted in a context that covers all or most of the facts. An example of a coherent fact set is “the game is a team sport”, “the game is played with a ball”, “the game demands great physical efforts”

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                        id2word=id2word,
                                                        num_topics=num_topics, 
                                                        chunksize=100,
                                                        passes=10,
                                                        per_word_topics=True,
                                                        workers=10)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
%%time
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=df['lemmas'], start=2, limit=22, step=4)

In [ ]:
# use these values for the plot if compute_coherence_values takes too long to run 
#coherence_values = [0.5054, 0.5332, 0.5452, 0.564, 0.5678, 0.5518, 0.519]

In [ ]:
limit=22; start=2; step=4;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
#optimal_model = model_list[1]
optimal_model =  models.LdaModel.load('optimal_model.model')
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))